<a href="https://colab.research.google.com/github/hikaru677/Frame-interpolation-dataset-creation/blob/main/calc_flow_magnitude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import random
import os
import ssl
import cv2
import numpy as np
import imageio
from IPython import display
from urllib import request
import re
import tempfile
import math
from tqdm.notebook import tqdm

Mounted at /content/drive


In [ ]:
! cd /content/drive

In [ ]:
! pip install torch
! pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install openmim
!mim install mmcv-full==1.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.8 MB/s 
     |████████████████████████████████| 237 kB 51.3 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 40.1 MB 1.4 MB/s 
     |████████████████████████████████| 190 kB 33.8 MB/s 


In [ ]:
! pip install mmflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 294 kB 29.6 MB/s 


In [ ]:
! mim download mmflow --config pwcnet_ft_4x1_300k_sintel_final_384x768

processing pwcnet_ft_4x1_300k_sintel_final_384x768...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 MiB 22.4 MB/s eta 0:00:00
Successfully downloaded pwcnet_ft_4x1_300k_sintel_final_384x768.pth to /root/.cache/mim
Successfully dumped pwcnet_ft_4x1_300k_sintel_final_384x768.py to /root/.cache/mim


In [ ]:
from mmflow.apis import init_model, inference_model
from mmflow.datasets import visualize_flow, write_flow
import mmcv
config_file = '/root/.cache/mim/pwcnet_ft_4x1_300k_sintel_final_384x768.py'
checkpoint_file = '/root/.cache/mim/pwcnet_ft_4x1_300k_sintel_final_384x768.pth'

# estimate

In [ ]:
# build the model from a config file and a checkpoint file
model = init_model(config_file, checkpoint_file, device='cuda:0')

# test image pair, and save the results
img1='/content/drive/MyDrive/UCF101_every_frame/Fixed/ApplyEyeMakeup/frames_v_ApplyEyeMakeup_g01_c01/000001.png'
img2='/content/drive/MyDrive/UCF101_every_frame/Fixed/ApplyEyeMakeup/frames_v_ApplyEyeMakeup_g01_c01/000002.png'
result = inference_model(model, img1, img2)
# save the optical flow file
write_flow(result, flow_file='flow.flo')
# # save the visualized flow map
flow_map = visualize_flow(result, save_file='flow_map.png')

load checkpoint from local path: /root/.cache/mim/pwcnet_ft_4x1_300k_sintel_final_384x768.pth


/usr/local/lib/python3.7/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
flow_map = cv2.readOpticalFlow("flow.flo")
condition1 = (flow_map < 0.01)
condition2 = (flow_map > -0.01)
condition = condition1 & condition2

flow_map[condition] = 0

flow_map_channel_0 = flow_map[:,:,0]
flow_map_channel_1 = flow_map[:,:,1]
print(flow_map.shape)

(240, 320, 2)


In [ ]:
flow_magnitude, _ = cv2.cartToPolar(flow_map_channel_0, flow_map_channel_1)

# A couple times, we've gotten NaNs out of the above...
nans = np.isnan(flow_magnitude)
if np.any(nans):
    nans = np.where(nans)
    flow_magnitude[nans] = 0.


False


In [ ]:
np.sqrt((flow_map_channel_0[0,0])**2 + (flow_map_channel_1[0,0])**2)

0.21235418672815243

In [ ]:
flow_magnitude

array([[0.21235418, 0.21235418, 0.19822504, ..., 0.31029785, 0.31658843,
        0.31658843],
       [0.21095131, 0.21095131, 0.19690162, ..., 0.31012207, 0.31643447,
        0.31643447],
       [0.19364929, 0.19364929, 0.18058035, ..., 0.3079982 , 0.3145934 ,
        0.3145934 ],
       ...,
       [0.08942665, 0.08942665, 0.08647271, ..., 0.20771086, 0.20938304,
        0.20938304],
       [0.09487536, 0.09487536, 0.09117741, ..., 0.20699666, 0.2077615 ,
        0.2077615 ],
       [0.09531953, 0.09531953, 0.09156311, ..., 0.2069289 , 0.20763001,
        0.20763001]], dtype=float32)

In [ ]:
np.count_nonzero(flow_magnitude > 3)

4178

In [ ]:
flow_map.shape[0] * flow_map.shape[1]# 240*320=76800

76800

In [ ]:
np.where(flow_magnitude > 3)

(array([ 61,  61,  61, ..., 150, 150, 150]),
 array([206, 207, 208, ..., 239, 240, 241]))

In [ ]:
flow_magnitude[61,207]

3.047536

In [ ]:
def flow_mag_stats(flow):
    """Get the average flow magnitude from a flow field.
    Args:
        flow: optical flow
    Returns:
        Average flow magnitude
    """
    # Convert the u,v flow field to angle,magnitude vector representation
    flow_magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # A couple times, we've gotten NaNs out of the above...
    nans = np.isnan(flow_magnitude)
    if np.any(nans):
        nans = np.where(nans)
        flow_magnitude[nans] = 0.

    return np.min(flow_magnitude), np.mean(flow_magnitude), np.max(flow_magnitude)


In [ ]:
def crop_square(frame):
    x, y = frame.shape[0:2]
    min_dim = min(x, y)
    start_x = 0
    start_y = 0
    return frame[start_x:start_x+min_dim, start_y:start_y+min_dim]

def load_video(path, resize=(64, 64)):
    cap = cv2.VideoCapture(path)
    max_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frames.append(frame)
            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)
